In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os


In [19]:
path = os.path.join(os.getcwd(),'data')
dfprod = pd.read_csv(os.path.join(path,'produccin-de-pozos-de-gas-y-petrleo-no-convencional.csv'), decimal='.')
dffrac = pd.read_csv(os.path.join(path,'datos-de-fractura-de-pozos-de-hidrocarburos-adjunto-iv-actualizacin-diaria.csv'), decimal='.')
dfprod['data'] = dfprod['fecha_data'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
dfprod['qo(kbpd)'] = 6.29*dfprod['prod_pet']/dfprod['fecha_data'].apply(lambda x:int(x[-2:]))
dfprod['qg(km3d)'] = dfprod['prod_gas']/dfprod['fecha_data'].apply(lambda x:int(x[-2:]))

In [7]:
dfprod['fecha_data']


0         2018-01-31
1         2015-01-31
2         2018-01-31
3         2017-01-31
4         2018-01-31
             ...    
311335    2023-12-31
311336    2023-12-31
311337    2023-12-31
311338    2023-12-31
311339    2023-12-31
Name: fecha_data, Length: 311340, dtype: object

In [9]:
dfprod = dfprod[dfprod['formacion']=='vaca muerta']
dfprod = dfprod[dfprod['tipoestado']!='Parado Transitoriamente']

In [11]:
dfprod2 = dfprod.drop([
    'anio',
    'mes',
    'tipoextraccion',
    'tipoestado',
    'tipopozo',
    'observaciones',
    'fechaingreso',
    'rectificado',
    'habilitado',
    'idusuario',
    'empresa',
    'sigla',
    'formprod',
    'profundidad',
    'formacion',
    'idareapermisoconcesion',
    'areapermisoconcesion',
    'idareayacimiento',
    'cuenca',
    'provincia',
    'coordenadax',
    'coordenaday',
    'tipo_de_recurso',
    'proyecto',
    'clasificacion',
    'subclasificacion',
    'sub_tipo_recurso',
    'fecha_data',
], axis=1)

In [12]:
idp = 162180
dfprod2[dfprod2['idpozo']==idp].sort_values('data').head()

,idempresa,idpozo,prod_pet,prod_gas,prod_agua,areayacimiento,data,qo(kbpd),qg(km3d)


In [13]:
datazero = dfprod2.groupby('idpozo')['data'].min()

In [14]:
dfprod3 = dfprod2.merge(right=datazero, how='left', on='idpozo', suffixes=('', '_start'))

dfprod3.head()

,idempresa,idpozo,prod_pet,prod_gas,prod_agua,areayacimiento,data,qo(kbpd),qg(km3d),data_start
0,YSUR,145330,0.0,0.0,0.0,HUACALERA,2018-01-31,0.0,0.0,2017-11-30
1,YSUR,145330,0.0,0.0,0.0,HUACALERA,2018-02-28,0.0,0.0,2017-11-30
2,YSUR,145330,0.0,0.0,0.0,HUACALERA,2018-03-31,0.0,0.0,2017-11-30
3,YSUR,145330,0.0,0.0,0.0,HUACALERA,2018-04-30,0.0,0.0,2017-11-30
4,YSUR,145330,0.0,0.0,0.0,HUACALERA,2018-05-31,0.0,0.0,2017-11-30


In [15]:
dfprod3['m'] = dfprod3['data'].dt.to_period('M').astype('int64') - dfprod3['data_start'].dt.to_period('M').astype('int64')

dfprod3[dfprod3['idpozo']==idp].sort_values('data').head()

,idempresa,idpozo,prod_pet,prod_gas,prod_agua,areayacimiento,data,qo(kbpd),qg(km3d),data_start,m


In [16]:
dffrac.columns

Index(['id_base_fractura_adjiv', 'idpozo', 'sigla', 'cuenca',
       'areapermisoconcesion', 'yacimiento', 'formacion_productiva',
       'tipo_reservorio', 'subtipo_reservorio', 'longitud_rama_horizontal_m',
       'cantidad_fracturas', 'tipo_terminacion', 'arena_bombeada_nacional_tn',
       'arena_bombeada_importada_tn', 'agua_inyectada_m3', 'co2_inyectado_m3',
       'presion_maxima_psi', 'potencia_equipos_fractura_hp',
       'fecha_inicio_fractura', 'fecha_fin_fractura', 'fecha_data', 'anio_if',
       'mes_if', 'anio_ff', 'mes_ff', 'anio_carga', 'mes_carga',
       'empresa_informante', 'mes', 'anio'],
      dtype='object')

In [17]:
dfprod4 = dfprod3.merge(right=dffrac[['idpozo','longitud_rama_horizontal_m','cantidad_fracturas']], how='left', on='idpozo')
cols = [
'idempresa',
'idpozo',
# 'prod_pet',
# 'prod_gas',
# 'prod_agua',
'areayacimiento',
'data',
'qo(kbpd)',
'qg(km3d)',
'data_start',
'm',
'longitud_rama_horizontal_m',
'cantidad_fracturas',
]

dfprod4 = dfprod4[cols]
dfprod4

,idempresa,idpozo,areayacimiento,data,qo(kbpd),qg(km3d),data_start,m,longitud_rama_horizontal_m,cantidad_fracturas
0,YSUR,145330,HUACALERA,2018-01-31,0.000000,0.000000,2017-11-30,2,NaN,NaN
1,YSUR,145330,HUACALERA,2018-02-28,0.000000,0.000000,2017-11-30,3,NaN,NaN
2,YSUR,145330,HUACALERA,2018-03-31,0.000000,0.000000,2017-11-30,4,NaN,NaN
3,YSUR,145330,HUACALERA,2018-04-30,0.000000,0.000000,2017-11-30,5,NaN,NaN
4,YSUR,145330,HUACALERA,2018-05-31,0.000000,0.000000,2017-11-30,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
8613,ACO,3640,EL CARACOL,2024-06-30,5.310976,0.158560,2023-03-31,15,NaN,NaN
8614,ACO,3640,EL CARACOL,2023-07-31,5.884565,0.302853,2023-03-31,4,NaN,NaN
8615,ACO,3640,EL CARACOL,2024-07-31,5.397611,0.208688,2023-03-31,16,NaN,NaN
8616,ACO,3640,EL CARACOL,2023-08-31,6.296665,0.299045,2023-03-31,5,NaN,NaN
